In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
import keras
from keras.layers import Embedding, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.utils import pad_sequences

In [ ]:
df = pd.read_csv("/content/cyberbullying_tweets.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [ ]:
df['cyberbullying_type'].unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [ ]:
def func(label):
  if label == 'not_cyberbullying':
    return 0
  else:
    return 1

In [ ]:
df['cyberbullying_type'] = df['cyberbullying_type'].apply(func)

In [ ]:
def clean(tweet):
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#\w+','',tweet)
    return tweet

df['tweet_text'] = df['tweet_text'].apply(clean)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
x_train = train_df['tweet_text']
y_train = train_df['cyberbullying_type']
x_test = test_df['tweet_text']
y_test = test_df['cyberbullying_type']


In [ ]:
tok = Tokenizer(num_words=42654, oov_token='[UNK]')
tok.fit_on_texts (x_train)
train_sq= tok.texts_to_sequences(x_train)
test_sq= tok.texts_to_sequences(x_test)
print (f'size of vocab is {len(tok.word_index)}')

size of vocab is 42654


In [ ]:
leng = df['tweet_text'].apply(len)

avg_len = leng.mean()

In [ ]:
avg_len

127.60639100897426

In [ ]:
max_sq_len= 110
vocab= len(tok.word_index)
embd_size= 130
train_sq= pad_sequences(train_sq, maxlen= max_sq_len, padding ='post', truncating='post')
test_sq= pad_sequences(test_sq, maxlen= max_sq_len, padding = 'post', truncating='post')

In [ ]:
from keras.layers import GlobalAveragePooling1D

model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=embd_size, input_length=max_sq_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(15, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_sq, y_train, epochs=10, validation_data=(test_sq, y_test))


Epoch 1/10
1193/1193 [==============================] - 92s 76ms/step - loss: 0.3461 - accuracy: 0.8416 - val_loss: 0.2914 - val_accuracy: 0.8465
Epoch 2/10
1193/1193 [==============================] - 85s 71ms/step - loss: 0.2601 - accuracy: 0.8645 - val_loss: 0.2769 - val_accuracy: 0.8479
Epoch 3/10
1193/1193 [==============================] - 87s 73ms/step - loss: 0.2329 - accuracy: 0.8828 - val_loss: 0.2804 - val_accuracy: 0.8488
Epoch 4/10
1193/1193 [==============================] - 87s 73ms/step - loss: 0.2127 - accuracy: 0.8952 - val_loss: 0.2926 - val_accuracy: 0.8502
Epoch 5/10
1193/1193 [==============================] - 88s 74ms/step - loss: 0.2003 - accuracy: 0.9028 - val_loss: 0.3054 - val_accuracy: 0.8442
Epoch 6/10
1193/1193 [==============================] - 88s 74ms/step - loss: 0.1892 - accuracy: 0.9062 - val_loss: 0.3211 - val_accuracy: 0.8470
Epoch 7/10
1193/1193 [==============================] - 91s 76ms/step - loss: 0.1794 - accuracy: 0.9119 - val_loss: 0.3416 -

In [ ]:

loss, accuracy = model.evaluate(test_sq, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


299/299 [==============================] - 1s 2ms/step - loss: 2.5962 - accuracy: 0.8298
Test Loss: 2.596158504486084
Test Accuracy: 0.8297515511512756
